<a href="https://colab.research.google.com/github/hanutm/Reinforcement-Learning/blob/master/Shortest_Path_RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pylab as plt
import networkx as nx
import pandas as pd

In [0]:
edge_list = [(0,2),(0,1),(0,3),(2,4),(5,6),(7,4),(0,6),(5,3),(3,7),(0,8),(3,4)]

In [0]:
goal = 7

In [0]:
G = nx.Graph()
G.add_edges_from(edge_list)

In [0]:
position = nx.spring_layout(G)

In [0]:
nx.draw_networkx_nodes(G,position)
nx.draw_networkx_edges(G,position)
nx.draw_networkx_labels(G,position)
plt.show()

In [0]:
#### Add Reward/ Q value matrices
size = 9

R = np.ones((size,size))
R *= -1
for i,j in edge_list:
  R[i,j]=0
  if i == goal or j == goal:
    R[i,j] = 100
R[goal,goal] = 100


Q = np.zeros((size,size))


Functions to include,



*   Find available actions; select specific action to perform
*   Update Q values



Find actions on the basis of the current state.
Actions possible only to states which have a reward >=0

In [0]:
def findActions(state):
  validActions = np.where(R[:,state] >= 0)[0]
  return validActions

2